In [ ]:
import os
import cv2
from file_dialog_utils import pick_images_cv2

import numpy as np

def rotate_and_central_crop(img, angle_deg, crop_factor=0.65, keep_size=False):
    """
    Rotate an upright template by angle_deg, then central-crop by crop_factor
    so that black corners from rotation are removed.

    Parameters
    ----------
    img : np.ndarray
        Input BGR image (upright template).
    angle_deg : float
        Rotation angle in degrees (positive = counter-clockwise).
    crop_factor : float
        How much of the centre region to keep (0 < crop_factor <= 1).
        e.g. 0.75 keeps 75% of width/height around the centre.
    keep_size : bool
        If True, resize result back to original (w, h).

    Returns
    -------
    out : np.ndarray
        Rotated + cropped (and optionally resized) BGR image.
    """
    
    h, w = img.shape[:2]
    center = (w // 2, h // 2)

    # 1) rotate, same canvas size
    M = cv2.getRotationMatrix2D(center, angle_deg, 1.0)
    rotated = cv2.warpAffine(
        img,
        M,
        (w, h),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,  # black corners appear here
        borderValue=(0, 0, 0)
    )

    # 2) central crop by crop_factor (zoom in)
    crop_w = int(w * crop_factor)
    crop_h = int(h * crop_factor)

    x1 = w // 2 - crop_w // 2
    y1 = h // 2 - crop_h // 2
    x2 = x1 + crop_w
    y2 = y1 + crop_h

    cropped = rotated[y1:y2, x1:x2]

    if keep_size:
        # 3) resize back to original size so templates are consistent
        cropped = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)

    return cropped


# Select one or more template images
templates_bgr, template_paths = pick_images_cv2(title="Select one or more template images")

# use numpy to generate float-stepped angles (range() doesn't accept float step)
angles = np.arange(-181.0, 181.0 + 0.1, 0.1) 

# Process each selected template
for template_bgr, template_path in zip(templates_bgr, template_paths):
    if template_bgr is None:
        raise ValueError(f"Template not found - check path: {template_path}")
    
    base = os.path.splitext(os.path.basename(template_path))[0]
    out_dir = f"C:\\Users\\Gabriel\\Documents\\Dissertation\\Code\\data\\{base}_rotated"
    os.makedirs(out_dir, exist_ok=True)

    for angle in angles:
        rotated = rotate_and_central_crop(template_bgr, float(angle))
        # format angle with sign and one decimal place for filename (e.g. +012.3)
        angle_str = f"{angle:+06.1f}"
        out_path = os.path.join(
            out_dir,
            f"{base}_{angle_str}.png"
        )
        cv2.imwrite(out_path, rotated)
        print("Saved", out_path)